In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [75]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
submission = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [76]:
train.shape

(42000, 785)

In [77]:
submission

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [78]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.src.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout,AveragePooling2D


In [80]:
def process_data(raw):
    out_y = to_categorical(raw.label, 10)
    number_of_images = raw.shape[0]
    images_array = raw.values[:, 1:]
    images_shaped_array = images_array.reshape(number_of_images, 28, 28, 1)
    out_x = images_shaped_array / 255
    return out_x, out_y

In [81]:
x, y = process_data(train)


Alright, we have splited our data in x and y, we have also reshaped and normalized it, it's time for model building

In [ ]:

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Conv2D(8, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=128,
          epochs=3,
          validation_split = 0.2)


Epoch 1/3
263/263 [==============================] - 44s 161ms/step - loss: 0.7931 - accuracy: 0.7367 - val_loss: 0.3129 - val_accuracy: 0.9025
Epoch 2/3
 30/263 [==>...........................] - ETA: 34s - loss: 0.2990 - accuracy: 0.9029

In [ ]:
def process_test(raw):
    number_of_images = raw.shape[0]
    images_array = raw.values[:, :]
    images_shaped_array = images_array.reshape(number_of_images, 28, 28, 1)
    out_x = images_shaped_array / 255
    return out_x

In [ ]:
test.shape

In [ ]:
x_test = process_test(test)
x_test.shape

In [ ]:
y_out = np.argmax(model.predict(x_test), axis=1)

In [ ]:
y_out

In [ ]:
submission = pd.DataFrame({"ImageId" : np.arange(1,test.shape[0] + 1).flatten(),"Label" :y_out}).to_csv("submission.csv",index=False)

In [ ]:
np.arange(1,test.shape[0] + 1).flatten()